## Set Up

In [1]:
import arcpy
from datetime import datetime
import os
import sys
from sqlalchemy.engine import URL
from sqlalchemy import create_engine
import sqlalchemy as sa
import pandas as pd
from arcgis import GIS
from arcgis.features import FeatureSet, GeoAccessor, GeoSeriesAccessor, FeatureLayer
import pandas as pd
import numpy as np
import requests


gis = GIS()
# # Set Pandas display options to show all rows and columns
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

# in memory output file path
wk_memory = "memory" + "\\"


# set workspace and sde connections 
#working_folder = "C:\GIS"
#working_folder = "F:/GIS/GIS_DATA/Monitoring/"
workspace      = "C:\GIS\Scratch.gdb"
#arcpy.env.workspace = "C:\GIS\Scratch.gdb"
#workspace ='F:\Research and Analysis\Workspace\Sarah\Data Management 2023\Scratch.gdb'
#arcpy.env.workspace = 'F:\Research and Analysis\Workspace\Sarah\Data Management 2023\Scratch.gdb'

# network path to connection files
filePath = "F:\\GIS\\DB_CONNECT"

#path to GDB's to update and master data
master_path = "F:\GIS\PROJECTS\ResearchAnalysis\SEZ\Data\SEZ_Data.gdb"
SEZ_Master = os.path.join(master_path, "AssessmentUnits_Master")

# database file paths 
### SDE Collection New data collected is put into SDE.Survey under the indicator name
### SDE Vector is where the data will go 
#sdeBase    = os.path.join(filePath, "Vector.sde")
sdeCollect = os.path.join(filePath, "Collection.sde")
sdeBase    = os.path.join(filePath, "SarahVector.sde")
#sdeCollect = os.path.join(filepath, "SarahCollect.sde")

# setup connection string???
connection_string = "DRIVER={ODBC Driver 17 for SQL Server};SERVER=sql12;DATABASE=sde;UID=sde;PWD=staff"
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
engine = create_engine(connection_url)

#with engine.begin() as sdeConnect:
 #   erosiondf      = pd.read_sql("SELECT * FROM sde.SDE.Stream_Erosion", sdeConnect)

# local variables sdata is starting data and f data is finishing datatables
ffdata = os.path.join(sdeBase, "sde.SDE.Monitoring")
sdata = os.path.join(sdeCollect, "sde.SDE.Survey")
fdata = os.path.join(master_path, "AssessmentUnits_Master")

##Tables we get the data from in Collect
sezsurveytable = os.path.join(sdata, "sde.SDE.sez_survey")
erosiondata = os.path.join(sdata, "sde.SDE.Stream_Erosion")
incisiondata = os.path.join(sdata, "sde.SDE.sez_channel_incision")
invasivedata = os.path.join(sdata, "sde.SDE.sez_invasive_plant")
headcutdata = os.path.join(sdata, "sde.SDE.sez_stream_headcut")
streamdata = os.path.join(ffdata, "sde.SDE.Stream")


#Staging Tables currently living in SEZ_Data.GDB
stage_bank_stability = os.path.join(master_path, "bank_stability") 
stage_All_SEZ_Scores = os.path.join(master_path, "All_SEZ_Scores")
stage_biotic_integrity = os.path.join(master_path, "biotic_integrity")
stage_headcuts_table = os.path.join(master_path, "heacuts_table")
stage_incision = os.path.join(master_path, "incision")
stage_invasives = os.path.join(master_path, "invasives")



#Final table to append to
#finalSEZtable = os.path.join(fdata, "sde.SDE.SEZ_Assessment_Unit")
#finalSEZtable = os.path.join(master_path, "AssessmentUnits_Master")


# network path to connection files??????
#filePath = "C:\\GIS\\DB_CONNECT"
# network path to connection files
#filePath = r'F:\Research and Analysis\Workspace\Sarah'

#--------------------------------------------#
#Notes to self
#--------------------------------------------#
#F:\GIS\PROJECTS\ResearchAnalysis\SEZ\Data\SEZ_Data.gdb. 
###'Assessment_Unit_Master' has all data for SEZ
###“Bioassessment Sample Locations” has stream sites in SEZs




## GDB with Raw Data straight from S123 not in the original folder (that one is not edited)
#headcut23gdb = os.path.join(working_folder,"Stream_Headcut","Stream_Headcut_Survey", "Stream_Headcut_Survey_2023.gdb")
#headcut22gdb = os.path.join(working_folder,"Stream_Headcut","Stream_Headcut_Survey", "Stream_Headcut_Survey_2022.gdb")
#headcut20gdb = os.path.join(working_folder,"Stream_Headcut","Stream_Headcut_Survey", "Stream_Headcut_Survey_2020.gdb")
#erosion23gdb = os.path.join(working_folder, "Stream_Erosion","Stream_Erosion_Survey", "Stream_Erosion_Survey_2023.gdb")
#erosion22gdb = os.path.join(working_folder, "Stream_Erosion","Stream_Erosion_Survey", "Stream_Erosion_Survey_2022.gdb")
#erosion20gdb = os.path.join(working_folder, "Stream_Erosion","Stream_Erosion_Survey", "Stream_Erosion_Survey_2020.gdb")
#channelincision23gdb = os.path.join(working_folder,"Channel_Incision","Channel_Incision_Survey","Channel_Incision_Survey_2023.gdb")
#channelincision22gdb = os.path.join(working_folder, "Channel_Incision","Channel_Incision_Survey","Channel_Incision_Survey_2022.gdb")
#channelincision20gdb = os.path.join(working_folder, "Channel_Incision","Channel_Incision_Survey","Channel_Incision_Survey_2020.gdb")
#invasiveplant23gdb= os.path.join(working_folder,"Invasive_Species","Invasive_Species_Survey","Invasive_Species_Survey_2023.gdb")
#invasiveplant22gdb= os.path.join(working_folder,"Invasive_Species","Invasive_Species_Survey","Invasive_Species_Survey_2022.gdb")
#invasiveplant20gdb= os.path.join(working_folder,"Invasive_Species","Invasive_Species_Survey","Invasive_Species_Survey_2020.gdb")

#This is thelocatoin for the final SEZ GDB to be updated in the gdb on f drive in the AssessmentUnits Master (polygon) i believe
#FinalGDBtoupdate:F:\GIS\PROJECTS\ResearchAnalysis\SEZ\Data \SEZ_Data.gdb

#Monitoring Dashboard location
#Finalsdelocation:f'Vector.SDE' Sde.Monitoring Sde. SEZ_Assessment_Unit

#Threshold Location? sde.tabular

## Lookup Dictionary for SEZ ID's


In [2]:
#-----------------------------------------------------------------------------------#
#Create look up dictionary for SEZ_ID fill in
#-----------------------------------------------------------------------------------#

# Set workspace environment
arcpy.env.workspace = master_path

# Specify the feature class name SEZ_Master
#feature_class = "AssessmentUnits_Master"

# Create a cursor to iterate over the rows in the feature class
fields = ['Assessment_Unit_Name', 'SEZ_ID']
data = []

with arcpy.da.SearchCursor(SEZ_Master, fields) as cursor:
    for row in cursor:
        data.append(row)

# Convert the data to a Pandas DataFrame
df = pd.DataFrame(data, columns=fields)

# Define function to update SEZ ID lookup dictionary
def update_SEZID_lookup_dict(df, lookup_dict):
    for index, row in df.iterrows():
        # Update SEZ_ID column in DataFrame with data from the lookup dictionary
        df.at[index, 'SEZ_ID'] = lookup_dict.get(row['Assessment_Unit_Name'], None)

    return df

# Assuming 'Assessment_Unit_Name' is the common identifier between DataFrame and lookup dictionary
selected_columns = ['Assessment_Unit_Name', 'SEZ_ID']

# Drop duplicates based on 'Assessment_Unit_Name' and keep the first occurrence
unique_values = df.drop_duplicates(subset='Assessment_Unit_Name', keep='first')

# Convert selected columns to dictionary
lookup_dict = unique_values[selected_columns].set_index('Assessment_Unit_Name').to_dict(orient='index')

# Display the dictionary
print(lookup_dict)

# Update 'SEZ_ID' in other DataFrame with data from the lookup dictionary do this for each indicator...
#Other_df = update_SEZID_lookup_dict(Other_df, lookup_dict)


{'Saxon Creek meadows - below Fountain Place': {'SEZ_ID': 1}, 'Saxon Creek meadows - above Fountain Place 1': {'SEZ_ID': 2}, 'Burke Creek meadows - 2': {'SEZ_ID': 3}, 'Washoe State Parks meadow - 1': {'SEZ_ID': 4}, 'Spooner Meadows - 4': {'SEZ_ID': 5}, 'Slaughterhouse Meadows - 1': {'SEZ_ID': 6}, 'Upper Truckee River - Tahoe Paradise': {'SEZ_ID': 7}, 'Hell Hole Meadows - 2': {'SEZ_ID': 8}, 'Trout Creek headwaters meadows - 2': {'SEZ_ID': 9}, 'small meadow 2': {'SEZ_ID': 10}, 'small meadow 1': {'SEZ_ID': 11}, 'North Logan House meadows': {'SEZ_ID': 12}, 'Logan House meadow': {'SEZ_ID': 13}, 'small meadow 59': {'SEZ_ID': 14}, 'Third Creek meadows - 8': {'SEZ_ID': 15}, 'Third Creek meadows - 7': {'SEZ_ID': 16}, 'small meadow 6': {'SEZ_ID': 17}, 'small meadow 15': {'SEZ_ID': 18}, 'small meadow 14': {'SEZ_ID': 19}, 'small meadow 13': {'SEZ_ID': 20}, 'small meadow 16': {'SEZ_ID': 21}, 'small meadow 54': {'SEZ_ID': 22}, 'small meadow 50': {'SEZ_ID': 23}, 'Freel Meadows - 2': {'SEZ_ID': 24}, '

In [ ]:
import pprint

# Display the dictionary using pprint
pprint.pprint(lookup_dict)


### Code for Grading each parameter

In [4]:
# Grading for each parameter add to final SDE file?
#Defining Grade for Bank Stability based on Erosiondf[percent_unstable]
def categorize_erosion(Percent_Unstable):
    if 0 <= value < 5:
        return 'A'
    elif 5 <= value < 20:
        return 'B'
    elif 20 <= value < 50:
        return 'C'
    else:
        return 'D'
    
#Scoring based off of grading - check this
def score_indicator(Rating):
    if  Rating = 'A':
        return '12'
    elif Rating = 'B':
        return '9'
    elif Rating = 'C':
        return '6'
    else:
        return '3'

#Define Grade for Incision based off of incisino ratio

def categorize_incision(bankful_ratio)
     if 0 <= bankful_ratio < 1.2:
        return 'A'
    elif 1.2 <= bankful_ratio < 1.6:
        return 'B'
    elif 1.6 <= bankful_ratio < 2.1:
        return 'C'
    else:
        return 'D'

#Define Grade for Bioassessment Score
def categorize_csci(biotic_integrity)
     if   biotic_integrity > 0.92:
        return 'A'
    elif 0.79 < biotic_integrity <= 0.92:
        return 'B'
    elif 0.62 < biotic_integrity <= 0.79:
        return 'C'
    else:
        return 'D'

#Define Grade for Invasive Plant Species Rating

def categorize_invasive(Invasive_Level)
      


def invasive_rating(grade)

#Define Size for Headcut based off of headcut size---fix this
##A = 0 headcut, B 1+small headcut
def categorize_headcut(value)
     if 0.1 <= value < 0.5:
        return 'small'
    elif 0.5 <= value < 1:
        return 'medium'
    else:
        return 'large'

#Define Size for Headcut based off of headcut size---fix this
##A = 0 headcut, B 1+small headcut
def categorize_headcutgrade(headcutgrade)
     if value = 0
        return 'A'
    elif 1.2 <= value < 1.6:
        return 'B'
    elif value =
        return 'C'
    else:
        return 'D'

#Define Grade for Invasive 
#A=0, B=1 level 2, C 2 level 2 or 1 level 1, D=3+ level 2 or 2+ level 1


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 29)

### Stream Erosion 

In [6]:
#---------------------------------------------------------------#
#GET DATA
#---------------------------------------------------------------#
#with engine.begin() as sdeConnect:
 #   erosiondf      = pd.read_sql("SELECT * FROM sde.SDE.Stream_Erosion", sdeConnect)

#counters = sdeBase + "\\sde.SDE.Transportation\\sde.SDE.bike_ped_counter_spatial"
#erosiondf = pd.DataFrame.spatial.from_featureclass(streamerosiontable)

#Convert feature class in gdb to Pandas DataFrame this is for REST Service
erosionfields = ['Assessment_Unit_Name', 'Shape.STLength()', 'Bank_Type', 'Percent_Unstable', 'Survey_Date']
#erosiondf = pd.DataFrame.from_records(data=arcpy.da.SearchCursor(erosiondata, erosionfields), columns=erosionfields)

# Initialize an empty list to store data
data = []

 # Iterate over the rows in the feature class and extract data
with arcpy.da.SearchCursor(erosiondata, erosionfields) as cursor:
    for row in cursor:
        data.append(row)

# Convert the data to a Pandas DataFrame
erosiondf = pd.DataFrame(data, columns=erosionfields)

#Add SEZ_ID column using lookup dictionary
#erosiondf['SEZ_ID'] = erosiondf['Assessment_Unit_Name'].map(lambda x: lookup_dict.get(x, None))


# Convert SEZ ID column to integer
#erosiondf['SEZ_ID'] = erosiondf['SEZ_ID'].astype(int)

erosiondf = erosiondf.merge(df, on='Assessment_Unit_Name', how='left')

#calculate year column 
erosiondf['Year'] = erosiondf['Survey_Date'].dt.year

#erosiondf['SEZ_ID'] = erosiondf['SEZ_ID'].astype(int)

#----------------------------------------------------------------#
#Process Data
#----------------------------------------------------------------#
#Make all data consistent capitalization and spaces
# Replace 'both_banks' with 'Both Banks' in Bank_Type column
erosiondf['Bank_Type'] = erosiondf['Bank_Type'].replace('both_banks', 'Both Banks', regex=True)
erosiondf['Bank_Type'] = erosiondf['Bank_Type'].replace(['one_bank', 'One bank'], 'One Bank', regex=True)
erosiondf['Bank_Type'] = erosiondf['Bank_Type'].replace(['no_bank', 'No bank'], 'No Bank', regex=True)

# Initialize variables
erosiondf['bank_multiplier'] = erosiondf['Bank_Type'].apply(lambda x: 2 if x == 'Both Banks' else (1 if x == 'One Bank' else 0))
erosiondf['Shape.STLength()'] = erosiondf['Shape.STLength()'].fillna(0)
erosiondf['bank_multiplier'] = erosiondf['bank_multiplier'].fillna(0)

# Calculate the product of 'Shape.STLength()' and 'bank_multiplier' to get the eroded banks per row
eroded_banks_per_row = erosiondf['Shape.STLength()'] * erosiondf['bank_multiplier']

# Group by Assessment_Unit_Name and sum the lengths of banks for each unit to get total banks assessed
banks_assessed_per_unit = erosiondf.groupby(['Assessment_Unit_Name', 'Year'])['Shape.STLength()'].sum()

# Group by Assessment_Unit_Name and sum the eroded banks per row for each unit
SEZ_total_eroded = erosiondf.groupby(['Assessment_Unit_Name', 'Year']).apply(lambda x: (x['Shape.STLength()'] * x['bank_multiplier']).sum())

# Reset index for banks_assessed_per_unit and SEZ_total_eroded
banks_assessed_per_unit.reset_index(drop=True, inplace=True)
SEZ_total_eroded.reset_index(drop=True, inplace=True)
banks_assessed_per_unit.dropna(inplace=True)
SEZ_total_eroded.dropna(inplace=True)


# Calculate percent unstable Bank Stability per Assessment Unit
erosiondf['Percent_Unstable'] = (SEZ_total_eroded / banks_assessed_per_unit) * 100


# Old Code Assuming erosiondf is your DataFrame containing the necessary data

# Initialize variables
#erosiondf['bank_multiplier'] = erosiondf['Bank_Type'].apply(lambda x: 2 if x == 'Both Banks' else (1 if x == 'One Bank' else 0))
#erosiondf['Shape.STLength()'] = erosiondf['Shape.STLength()'].fillna(0)
#erosiondf['bank_multiplier'] = erosiondf['bank_multiplier'].fillna(0)
# Calculate the product of 'Shape.STLength()' and 'bank_multiplier' to get the eroded banks per row
#eroded_banks_per_row = erosiondf['Shape.STLength()'] * erosiondf['bank_multiplier']

# Group by Assessment_Unit_Name and sum the lengths of banks for each unit to get total banks assessed
#banks_assessed_per_unit = erosiondf.groupby('Assessment_Unit_Name')['Shape.STLength()'].sum()

# Group by Assessment_Unit_Name and sum the eroded banks per row for each unit
#SEZ_total_eroded = erosiondf.groupby('Assessment_Unit_Name')[eroded_banks_per_row].sum()
# Group by Assessment_Unit_Name and sum the eroded banks per row for each unit
#SEZ_total_eroded = erosiondf.groupby('Assessment_Unit_Name')[erosiondf['Shape.STLength()'] * erosiondf['bank_multiplier']].sum()

# Calculate percent unstable Bank Stability per Assessment Unit
#percent_unstable_bank_stability = (SEZ_total_eroded / banks_assessed_per_unit) * 100

# Assign the calculated percent unstable bank stability to a new column in the DataFrame
#erosiondf['Percent_Unstable'] = percent_unstable_bank_stability

#erosiondf['Percent_Unstable'] = (SEZ_total_eroded/ banks_assessed_per_unit) * 100
# Field Mapping
#field_mapping = {
 #   'StationCod': 'SITE_NAME',
  #  'STATION_CODE': 'STATION_CODE',
   # 'Year': 'YEAR_OF_COUNT',
    #'LATITUDE': 'LATITUDE',
    #'LONGITUDE': 'LONGITUDE',
    #'CSCI': 'COUNT_VALUE',
    #'STATION_TYPE': 'STATION_TYPE',
    #'LTINFO': 'LTINFO'
#}

# Rename fields based on field mappings
#ready_df = filtered_df.rename(columns=field_mapping).drop(columns=[col for col in filtered_df.columns if col not in field_mapping])

# Combine the calculated values into a new DataFrame
#summary_df = pd.DataFrame({
 #   'Total_Banks_Assessed_per_SEZ': banks_assessed_per_sez,
  #  'Total_Eroded_Banks_per_SEZ': eroded_banks_per_sez,
   # 'Percent_Unstable_per_SEZ': percent_unstable_per_sez
#})

# Print the summary DataFrame
#print("Summary DataFrame:")
#print(summary_df)



print(erosiondf)

#----------------------------------------------------------------#
#Grade, Score, finish data frame
#----------------------------------------------------------------#
#erosiondf['Bank_Stability_Rating']=erosiondf['Percent_Unstable].apply.(categorize_erosion)
#erosiondf['Bank_Stability_Score']= erosiondf['Bank_Stability_Score].apply(score_indicator)
#erosiondf['Year'] = erosiondf.Survey_Date.str.split("_").str.[-1]????
#erosiondf['Bank_Stability_Data_Source']= 'TRPA'
#----------------------------------------------------------------#
#post ending dataframe to bank_stability called stage_bank_stability
#----------------------------------------------------------------#


                              Assessment_Unit_Name  Shape.STLength()  \
0                         Angora Creek - tributary         13.800945   
1                         Angora Creek - tributary         13.800945   
2                         Angora Creek - tributary         88.188503   
3                         Angora Creek - tributary         88.188503   
4                         Angora Creek - tributary       1582.500924   
...                                            ...               ...   
4179                                Big Meadow - 4        790.825100   
4180                  Trout Creek below Black Bart        108.820413   
4181                  Trout Creek below Black Bart        108.820413   
4182  Saxon Creek meadows - above Fountain Place 2         71.738302   
4183  Saxon Creek meadows - above Fountain Place 2         71.738302   

     Bank_Type  Percent_Unstable                Survey_Date  SEZ_ID  Year  \
0     One Bank          1.718680 2023-05-24 09:49:00.00000

In [18]:

# Print SEZ_total_eroded
print("SEZ_total_eroded:")
print(SEZ_total_eroded)

# Print banks_assessed_per_unit
print("\nbanks_assessed_per_unit:")
print(banks_assessed_per_unit)

print(erosiondf['Percent_Unstable'])

SEZ_total_eroded:
Assessment_Unit_Name
 Glenbrook creek - middle     40.148192
Angora Creek - tributary      70.293909
Angora meadows - 1             0.000000
Angora meadows - 2             0.000000
Angora meadows - 3           255.862258
                                ...    
small meadow 51                0.000000
small meadow 52                0.000000
small meadow 55                0.000000
small meadow 7                62.050020
small meadow 95               37.664963
Length: 219, dtype: float64

banks_assessed_per_unit:
Assessment_Unit_Name
 Glenbrook creek - middle      40.148192
Angora Creek - tributary     4089.993846
Angora meadows - 1           2784.726750
Angora meadows - 2           3142.696365
Angora meadows - 3           2585.982713
                                ...     
small meadow 51               126.910853
small meadow 52               402.107242
small meadow 55               349.755802
small meadow 7               2448.624297
small meadow 95               954.24

In [11]:
# Convert DataFrame to CSV
#csv_path = os.path.join(working_folder, "OldCSCIdata.csv")
#ready_df.to_csv(csv_path, index=False) 

[  nan 15.32]


In [ ]:


# Input feature class 
input_fc = 'Stream_Erosion'

# Convert feature class to Pandas DataFrame
erosionfields = ['Assessment_Unit_Name', 'Shape.STLength()', 'Bank_Type', 'Percent_Unstable',]
erosiondf = pd.DataFrame.from_records(data=arcpy.da.SearchCursor(input_fc, erosionfields), columns=erosionfields)

# Initialize variables
erosiondf['bank_multiplier'] = erosiondf['Bank_Type'].apply(lambda x: 2 if x == 'Both Banks' else (1 if x == 'One Bank' else 0))

# Calculate the total length of banks assessed
total_banks_assessed = (erosiondf['Shape_Length'].sum()) * 2

# Calculate the total length of eroded banks
total_eroded_banks = (erosiondf['Shape_Length'] * erosiondf['bank_multiplier']).sum()

# calculate percent unstable 'percent_unstable' column!!! Bank Stability and we will need to add a column to the dataframe
erosiondf['percent_unstable'] = (total_eroded_banks / total_banks_assessed) * 100

print("Total Banks Assessed:", total_banks_assessed)
print("Total Eroded Banks:", total_eroded_banks)
print("Updated DataFrame:")
print(erosiondf)

#field mapping? what is the fi nal location
#field_mapping ={

    #'Assessment_Unit_Name': 
    #'Shape_Length'
    #"Bank_Type"
    #'Percent_Unstable'
#}

# Create a new DataFrame Final_df with columns based on the field mapping
#df = df.rename(columns=field_mapping)

# Convert DataFrame to Spatially Enabled DataFrame
sedf = GeoAccessor.from_xy(erosiondf, x_column='LONGITUDE', y_column='LATITUDE')

# Convert the SEDF to a feature class without sanitizing columns where do i want this to live... its all of the gdb's combined so where do i update it.. to each gdb or individually or create a new one or put a copy into scratch.gdb and then some how write back 
sedf.spatial.to_featureclass(location=os.path.join(workspace, 'Erosion_Staging'), sanitize_columns=False)


### Channel Incision

In [ ]:
#Channel Incision Process Data

# List of paths to geodatabases
channelincisiongdbs = ['channelincision20gdb', 'channelincision22gdb', 'channelincision23gdb']

# Initialize an empty list to store DataFrames
channeldfs = []

# Iterate over each geodatabase
for gdb in channelincisiongdbs:
    # List feature classes or tables in the geodatabase
    feature_classes = arcpy.ListFeatureClasses("*", "All", gdb)
    tables = arcpy.ListTables("*", "All", gdb)
    
    # Iterate over feature classes and tables
    for fc in feature_classes + tables:
        # Convert feature class or table to DataFrame
        df = pd.DataFrame(arcpy.da.FeatureClassToNumPyArray(fc, '*'))
        dhanneldfs.append(df)

# Concatenate all DataFrames into a single DataFrame
erosion_df = pd.concat(channeldfs)

#Processing of Channel Incision Data

### Invasive Species

In [ ]:
#Invasive Species

### HEadcuts


In [ ]:
#Headcuts

### Bioassessment/ Biotic Integrity

In [ ]:
#Bioassessment scores- get all stream data into sde.Stream first... then look at Biotic Integrit Data Source to find which stream was used to evaluate each meadow.. this will help with percent of stream miles I believe

In [ ]:
#Conifer Encroachment

In [ ]:
#Aquatic Organism /Fish Passage

##### Code to make it a spatially enabled dataframe

In [ ]:
#code to conver to spatially enable geodataframe
# Create a new DataFrame Final_df with columns based on the field mapping
#df = df.rename(columns=field_mapping)

# Convert DataFrame to Spatially Enabled DataFrame
#sedf = GeoAccessor.from_xy(df, x_column='LONGITUDE', y_column='LATITUDE')

# Convert the SEDF to a feature class without sanitizing columns
#sedf.spatial.to_featureclass(location=os.path.join(workspace, 'PlanAreaNoise_23_Staging'), sanitize_columns=False)